# Prática Independente

# Setup

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import auc, precision_recall_curve, log_loss,confusion_matrix, recall_score, precision_score
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings('ignore')


# Dados

In [2]:
df_churn = pd.read_csv('https://raw.githubusercontent.com/abnr/ml-data/main/churn.csv')

Realizem a divisão dos dados em Treino e Teste, com 20% dos dados para teste e o parâmetro ```random_state``` = 123

In [3]:
# digite seu código aqui
df_train, df_test = train_test_split(df_churn, 
                                     random_state=123, 
                                     test_size=0.2,
                                     stratify=df_churn['Churn'])

In [4]:
X_train, y_train = df_train.drop('Churn', axis=1), df_train['Churn']
X_test, y_test = df_test.drop('Churn', axis=1), df_test['Churn']

## Divisão dos Dados

Divida seus dados de forma estratificada ```random_state=123``` e separe as colunas categóricas das numéricas em duas listas.

In [5]:
cat_columns_churn = X_train.select_dtypes(include=['object']).columns.tolist()
num_columns_churn = X_train.select_dtypes(exclude=['object']).columns.tolist()

In [6]:
cat_columns_churn

['State', 'International plan', 'Voice mail plan']

In [7]:
num_columns_churn

['Account length',
 'Area code',
 'Number vmail messages',
 'Total day minutes',
 'Total day calls',
 'Total day charge',
 'Total eve minutes',
 'Total eve calls',
 'Total eve charge',
 'Total night minutes',
 'Total night calls',
 'Total night charge',
 'Total intl minutes',
 'Total intl calls',
 'Total intl charge',
 'Customer service calls']

# Pipelines

Crie um pipeline para processamento dos dados de entrada. Primeiro crie um pipeline categórico com ```OneHotEncoder``` e depois um pipeline numérico com ```MinMaxScaler```

Combine tudo com o ```ColumnTransformer```

In [8]:
pipe_cat_columns_churn = ('one_hot_encoder', 
                          OneHotEncoder(handle_unknown='ignore'),
                          cat_columns_churn)

pipe_num_columns_churn = ('min_max_scaler',
                          MinMaxScaler(), 
                          num_columns_churn)

In [9]:
transformers = [pipe_num_columns_churn, pipe_cat_columns_churn]
pre_processor = ColumnTransformer(transformers)

# Modelagem

Compare os seguintes algoritmos, utilizando a métrica log loss.:
- Regressão Logística
- SVM
- KNN
- Decision Tree
- Random Forest

Construa um pipeline para cada um, utilizando o ```ColumnTransformer``` do passo anterior.

In [10]:
model_reglog = Pipeline(steps=[('pre_processor', pre_processor), 
                               ('model', LogisticRegression(max_iter=5000, 
                                                            random_state=123))])
# pipeline para SVM
model_svm = Pipeline(steps=[('pre_processor', pre_processor), 
                               ('model', SVC(kernel='rbf', probability=True))])
# pipeline para KNN

Vamos rodar todos os modelos e checar a performance com alguma métrica a sua escolha. Dica, crie uma lista de modelos e rode um for loop.

In [11]:
classifiers = [model_reglog, model_svm]

for classifier in classifiers:
    classifier.fit(X_train, y_train)
    print(type(classifier['model']).__name__,'\n')
    print(f'log_loss treino: {log_loss(y_train, classifier.predict_proba(X_train)[:,1])}')
    print(f'log_loss teste: {log_loss(y_test, classifier.predict_proba(X_test)[:,1])}')
    print(100*'-')

LogisticRegression 

log_loss treino: 0.3133204901958391
log_loss teste: 0.3211824543598783
----------------------------------------------------------------------------------------------------
SVC 

log_loss treino: 0.23463365734356767
log_loss teste: 0.2983980374844398
----------------------------------------------------------------------------------------------------


# Avaliação do Threshold

Treine o modelo campeão.

In [ ]:
#digite seu código aqui

Avalie as métricas em relação ao threshold escolhido.

In [ ]:
#digite seu código aqui

Podemos ver esse problema de uma outra forma. E se em vez de responder a esse tipo de pergunta, quiséssemos escolher o threshold que permitisse que a empresa lucrasse mais?

Crie uma função de custo personalizada para o problema, levando em conta falsos positivos, falsos negativos, verdadeiros positivos e verdadeiros negativos e escolha o threshold que maximiza ou minimiza essa função (dependendo do problema). Qual o impacto financeiro de um Falso Positivo? E de um Falso Negativo? E de um Verdadeiro Positivo? E de um Verdadeiro Negativo?

Para esse exercício, precisamos definir os custos e receita de cada componente da matriz de confusão.

**Verdadeiro Positivo:** O verdadeiro positivo é o cara que iria desativar o produto e foi identificado corretamente pelo modelo. Esse cara gasta em média R$ 150 por mês com o produto da empresa e tem uma probabilidade de 15 por cento de ser retido, caso liguem para ele. 

 

Dessa forma podemos criar uma expressão que traz em média o "custo" de um Verdadeiro Positivo. Lembrando que a ligação também tem um custo, que vamos admitir que seja de R$ 10,00. Dessa forma, a expressão para o Verdadeiro positivo fica:    
     VP = 0.15*150 - 10



**Falso Positivo**: O falso positivo insere o custo da ligação. Dessa forma  a expressão fica:
FP = 150 - 10

**Verdadeiro Negativo**: O verdadeiro negativo traz lucro para a empresa, pois consome em média 150 reais.<br>
VN = 150

**Falso Negativo**: Por fim, o Falso Negativo é o cara que simplesmente deixa o produto e não transaciona mais, vamos admitir que é uma perda para a empresa. Como cada cliente transaciona em média 150 reais, a empresa terá uma perda equivalente.
<br>
FN = -150

In [ ]:
#digite seu código aqui